In [ ]:
# Jersey Order Management For Youth Hockey Teams

## Overview

- This project is a web application that allows youth hockey teams to manage their jersey orders.  
- It centers around a jupyter notebook that is used to manage the jersey orders.

## Features

- Add a new jersey order
- View all jersey orders
- Update a jersey order
- Delete a jersey order